## RFM Model

The RFM model is a customer segmentation technique that analyzes Recency, Frequency, and Monetary value to categorize customers based on their recent purchase behavior, how often they make purchases, and the monetary value of their transactions. It is commonly used in marketing and e-commerce to identify and target specific customer segments for tailored strategies.

The RFM model is a customer segmentation approach that evaluates three key aspects:
    

 * <b>Recency (R): How recently a customer made a purchase.
    

 * <b>Frequency (F): How often a customer makes purchases.
    

 * <b>Monetary Value (M): The total monetary value of a customer's purchases.
    

By analyzing these three factors, businesses can categorize customers into segments, allowing for targeted marketing and engagement strategies based on their past behavior.

## Step 1 : - Data Import and Library¶

In [8]:
# Import libraries
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
import squarify
import seaborn as sns

In [14]:
df=pd.read_csv(r'data.csv',encoding='latin-1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [18]:
df.shape

(541909, 8)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [16]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

## Step 2 :- Data Pre-Processing¶

In [20]:
df = df.dropna()

In [21]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [26]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])  # yyyy-mm-dd
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Identify the unique customer

In [27]:
len(list(df['CustomerID'].unique()))

4372

In [28]:
print('{:,} rows; {:,} columns'
      .format(df.shape[0], df.shape[1]))

print('{:,} transactions don\'t have a customer id'
      .format(df[df.CustomerID.isnull()].shape[0]))

print('Transactions timeframe from {} to {}'.format(df['InvoiceDate'].min(),
                                    df['InvoiceDate'].max()))

406,829 rows; 8 columns
0 transactions don't have a customer id
Transactions timeframe from 2010-12-01 08:26:00 to 2011-12-09 12:50:00


## Step 3 : - Model Building RFM

In [29]:
# Create snapshot date
snapshot_date = df['InvoiceDate'].max() + timedelta(days=1)
print(snapshot_date)

2011-12-10 12:50:00


In [30]:
# Grouping by CustomerID
data_process = df.groupby(['CustomerID']).agg({
        'InvoiceDate': [lambda x: (snapshot_date - x.max()).days, 'count'],
        'UnitPrice': 'sum'})

In [31]:
data_process

InvoiceDate       UnitPrice
            <lambda_0> count       sum
CustomerID                            
12346.0            326     2      2.08
12347.0              2   182    481.21
12348.0             75    31    178.71
12349.0             19    73    605.10
12350.0            310    17     65.30
...                ...   ...       ...
18280.0            278    10     47.65
18281.0            181     7     39.36
18282.0              8    13     62.68
18283.0              4   756   1220.93
18287.0             43    70    104.55

[4372 rows x 3 columns]

In [32]:
# Rename the columns 
data_process.rename(columns={'InvoiceDate': 'Recency',
                             'count': 'Frequency',
                             'UnitPrice': 'MonetaryValue'}, inplace=True)

data_process

Recency           MonetaryValue
           <lambda_0> Frequency           sum
CustomerID                                   
12346.0           326         2          2.08
12347.0             2       182        481.21
12348.0            75        31        178.71
12349.0            19        73        605.10
12350.0           310        17         65.30
...               ...       ...           ...
18280.0           278        10         47.65
18281.0           181         7         39.36
18282.0             8        13         62.68
18283.0             4       756       1220.93
18287.0            43        70        104.55

[4372 rows x 3 columns]

In [36]:
data_process.columns=data_process.columns.droplevel()

In [39]:
# Rename the columns 
data_process.rename(columns={'<lambda_0>': 'Recency',
                         'count': 'Frequency',
                         'sum': 'MonetaryValue'}, inplace=True)

data_process

,Recency,Frequency,MonetaryValue
CustomerID,,,
12346.0,326,2,2.08
12347.0,2,182,481.21
12348.0,75,31,178.71
12349.0,19,73,605.10
12350.0,310,17,65.30
...,...,...,...
18280.0,278,10,47.65
18281.0,181,7,39.36
18282.0,8,13,62.68


In [40]:
# Print top 5 rows and shape of dataframe
print(data_process.head())
print('{:,} rows; {:,} columns'.format(data_process.shape[0], data_process.shape[1]))

            Recency  Frequency  MonetaryValue
CustomerID                                   
12346.0         326          2           2.08
12347.0           2        182         481.21
12348.0          75         31         178.71
12349.0          19         73         605.10
12350.0         310         17          65.30
4,372 rows; 3 columns


In [41]:
#--Calculate R and F groups--
# Create labels for Recency  
r_labels = range(4, 0, -1)  # 4 , 3 , 2 , 1 

# Create labels for Frequency

f_labels = range(1, 5)      # 1 , 2 ,3 ,4 

# Create labels for MonetaryValue
m_labels = range(1, 5)  # 1 , 2 ,3 ,4     # 


# Assign these labels to 4 equal percentile groups 
r_groups = pd.qcut(data_process['Recency'], q=4, labels=r_labels)


# Assign these labels to 4 equal percentile groups 
f_groups = pd.qcut(data_process['Frequency'], q=4, labels=f_labels)

# Assign these labels to three equal percentile groups 
m_groups = pd.qcut(data_process['MonetaryValue'], q=4, labels=m_labels)


# Create new columns R and F 
data_process = data_process.assign(R = r_groups.values, F = f_groups.values, M = m_groups.values)


data_process.head()

,Recency,Frequency,MonetaryValue,R,F,M
CustomerID,,,,,,
12346.0,326,2,2.08,1,1,1
12347.0,2,182,481.21,4,4,4
12348.0,75,31,178.71,2,2,3
12349.0,19,73,605.10,3,3,4
12350.0,310,17,65.30,1,1,2


In [42]:
data_process['RFM_Segment_Concat'] = data_process.R.astype(str) + data_process.F.astype(str) + data_process.M.astype(str)
rfm = data_process
rfm.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment_Concat
CustomerID,,,,,,,
12346.0,326,2,2.08,1,1,1,111
12347.0,2,182,481.21,4,4,4,444
12348.0,75,31,178.71,2,2,3,223
12349.0,19,73,605.10,3,3,4,334
12350.0,310,17,65.30,1,1,2,112


In [43]:
# Count num of unique segments
rfm_count_unique = rfm.groupby('RFM_Segment_Concat')['RFM_Segment_Concat'].nunique()
print(rfm_count_unique.sum())

56


In [44]:
# Calculate RFM_Score
rfm['RFM_Score'] = rfm[['R','F','M']].sum(axis=1)
rfm.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment_Concat,RFM_Score
CustomerID,,,,,,,,
12346.0,326,2,2.08,1,1,1,111,3
12347.0,2,182,481.21,4,4,4,444,12
12348.0,75,31,178.71,2,2,3,223,7
12349.0,19,73,605.10,3,3,4,334,10
12350.0,310,17,65.30,1,1,2,112,4


In [46]:
# Define rfm_level function
def rfm_level(df):
    if df['RFM_Score'] >= 10:
        return 'Loyal / Potential / Can\'t Loose Them'
    elif ((df['RFM_Score'] >= 5) and (df['RFM_Score'] < 10)):
        return 'Champions / Promising '
    elif ((df['RFM_Score'] >= 4) and (df['RFM_Score'] < 5)):
        return 'Needs Attention'
    else:
        return 'Require Activation'

In [47]:
rfm['Customer_Segment'] = rfm.apply(rfm_level, axis=1)
rfm.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment_Concat,RFM_Score,Customer_Segment
CustomerID,,,,,,,,,
12346.0,326,2,2.08,1,1,1,111,3,Require Activation
12347.0,2,182,481.21,4,4,4,444,12,Loyal / Potential / Can't Loose Them
12348.0,75,31,178.71,2,2,3,223,7,Champions / Promising
12349.0,19,73,605.10,3,3,4,334,10,Loyal / Potential / Can't Loose Them
12350.0,310,17,65.30,1,1,2,112,4,Needs Attention


In [48]:
# Calculate average values for each RFM_Level, and return a size of each segment 
rfm_level_agg = rfm.groupby('Customer_Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count']
}).round(1)
rfm_level_agg

Recency Frequency MonetaryValue      
                                        mean      mean          mean count
Customer_Segment                                                          
Champions / Promising                   93.0      42.5         160.1  2265
Loyal / Potential / Can't Loose Them    20.0     233.7         788.5  1295
Needs Attention                        144.2      11.8          37.0   398
Require Activation                     262.2       7.4          22.8   414

In [49]:
rfm_level_agg.columns = rfm_level_agg.columns.droplevel()
rfm_level_agg.columns = ['RecencyMean','FrequencyMean','MonetaryMean', 'Count']
rfm_level_agg

,RecencyMean,FrequencyMean,MonetaryMean,Count
Customer_Segment,,,,
Champions / Promising,93.0,42.5,160.1,2265
Loyal / Potential / Can't Loose Them,20.0,233.7,788.5,1295
Needs Attention,144.2,11.8,37.0,398
Require Activation,262.2,7.4,22.8,414


_________________________________________________________________________________________

__________________________________________________________________________

___________________________________________________________________________________

______________________________________________________________________________________

________________________________________________________________________________________

____________________________________________________________________________________________

__________________________________________________________________________________

# Conclusion

### Percentage of Loyal Customers

In [51]:
1295/(2265+1295+398+414)*100 

29.620311070448306

### Loyal Customer Transaction amount in %

In [54]:
(788.5 * 1295)*100/((788.5 * 1295) +(160.1*2265)+(37.0*398)+(22.8*414))

72.52703176477407

### 30 % of Loyal Customers generate 73 % of Revenue

Personalized Marketing:

Implement personalized marketing campaigns targeted specifically at this segment. Use their past purchase history and preferences to tailor promotions, discounts, and product recommendations.
Exclusive Offers and Rewards:


Provide exclusive offers, discounts, or loyalty rewards to incentivize repeat purchases. Make these rewards valuable and exclusive to reinforce their status as valued customers.
VIP Programs:


Consider establishing a VIP or loyalty program specifically for this segment. Offer special perks, early access to new products, or premium customer service to enhance their experience